In [1]:
import pickle

In [2]:
with open('final_result.pkl', 'rb') as f:
    data = pickle.load(f)

In [12]:
len(data)

2717

In [3]:
N = 5  
for i, (key, value) in enumerate(data.items()):
    print(key)
    if i + 1 == N:  
        break


[가입절차] 스마트스토어센터 회원가입은 어떻게 하나요? (ID만들기)
[가입서류] 스마트스토어 판매자 유형별 필요한 서류가 어떻게 되나요?
[심사결과] 가입 신청을 했습니다. 심사 결과는 어떻게 확인하나요?
[가입절차][쇼핑윈도/패션타운] 네이버 쇼핑윈도 노출 절차는 어떻게 되나요?
[가입서류] 국내 사업자로 가입하려면 어떤 서류가 필요한가요?


In [4]:
N = 5  
for i, (key, value) in enumerate(data.items()):
    print(value)
    if i + 1 == N:  
        break


네이버 커머스 ID 하나로 스마트스토어센터와 같은 네이버의 다양한 커머스 서비스를 편리하게 이용하실 수 있습니다.네이버 커머스 ID가 없으시다면 [가입하기] 버튼을 통해 회원가입해 주세요.           1. '네이버ID' 로 네이버 커머스 ID 가입하기1) [네이버 아이디로 가입하기] 버튼을 눌러서 네이버 로그인 해 주세요.2) 연락가능한 휴대폰번호를 입력하시고 인증하신 후 개인정보 수집동의를 체크 하여 [가입] 버튼을 눌러주세요. 3) 네이버 커머스 ID 회원가입이 완료되어 해당 네이버ID 로 스마트스토어센터 서비스 가입을 하실 수 있습니다. 이 후 스마트스토어센터 로그인 시에 [네이버 아이디로 로그인] 버튼을 눌러서 네이버 로그인을 하실 수 있습니다. 2. '이메일 아이디'로 네이버 커머스 ID 가입하기 1) [이메일 아이디로 가입하기] 버튼을 눌러서 사용할 ID를 입력해 주세요.! 중요. 로그인 ID는 "실 사용중인 이메일주소"로 기재하셔야 합니다. (예 : abc@naver.com)2) 연락가능한 휴대폰번호와 이메일주소를 입력하시고 인증하신 후 개인정보 수집동의를 체크 하여 [가입] 버튼을 눌러주세요.3) 네이버 커머스 ID 회원가입이 완료되어 해당 이메일 아이디로 스마트스토어센터 서비스 가입을 하실 수 있습니다.이 후 스마트스토어센터 로그인 시에 해당 이메일 아이디와 비밀번호를 입력하여 로그인을 하실 수 있습니다. ※ 참고. 로그인 ID 계정 대표자와 사업자번호 대표자 동일여부는 체크하지 않습니다. (대표자가 상이해도 가입 진행 가능합니다.)             


위 도움말이 도움이 되었나요?


별점1점

별점2점

별점3점

별점4점

별점5점



소중한 의견을 남겨주시면 보완하도록 노력하겠습니다.

보내기



관련 도움말/키워드

스마트스토어 로그인ID(매니저)를 추가하거나 변경할 수 없나요?
네이버 커머스 ID 전환 이후, 이전 아이디로 로그인이 불가한가요?
네이버 커머스 ID 전환 후 로그인이 되지 않아요.



도움말 닫기
스

In [9]:
pip install openai


   ---------------------------------------- 0.0/267.1 kB ? eta -:--:--
   ---------------------------------------  266.2/267.1 kB 8.0 MB/s eta 0:00:01
   ---------------------------------------- 267.1/267.1 kB 5.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/75.6 kB ? eta -:--:--
   ---------------------------------------- 75.6/75.6 kB ? eta 0:00:00
   ---------------------------------------- 0.0/77.9 kB ? eta -:--:--
   ---------------------------------------- 77.9/77.9 kB ? eta 0:00:00


In [30]:
import pickle
import openai
from openai import OpenAI
from math import ceil
from tqdm import tqdm

# Load your FAQ data
with open('final_result.pkl', 'rb') as file:
    faq_data = pickle.load(file)
    
client = OpenAI(api_key='')

# Assuming faq_data is a dictionary {question: answer}
# Convert it into a list of concatenated question-answer pairs for embedding
faq_pairs = [q + " " + a for q, a in faq_data.items()]

# Function to divide data into batches
def divide_into_batches(data, batch_size):
    for i in range(0, len(data), batch_size):
        yield data[i:i + batch_size]

# Function to process a single batch
def process_batch(batch):
    embeddings = []
    for text in tqdm(batch, desc="Processing batch", leave=False):  # Wrap batch in tqdm
        response = client.embeddings.create(
            input=text,
            model="text-embedding-3-small"  # Adjust model as needed
        )
        embedding = response.data[0].embedding
        embeddings.append(embedding)
    return embeddings

# Main processing function
def get_embeddings(faq_pairs, batch_size=5):
    all_embeddings = []
    batches = divide_into_batches(faq_pairs, batch_size)
    for batch in tqdm(batches, desc="Processing batches", leave=True):  # Wrap batches in tqdm
        batch_embeddings = process_batch(batch)
        all_embeddings.extend(batch_embeddings)
    return all_embeddings
    
# Example usage
batch_size = 20  # Choose based on OpenAI's limitations and your needs
embeddings = get_embeddings(faq_pairs, batch_size=batch_size)

# The list of dictionaries where each dictionary represents a sentence, 
# with the 'embeddings' key containing the corresponding vector embedding and the 'text' key containing the original sentence
sentences_list = [
    {
        'embeddings': vector,
        'text': sentence
    }
    for vector, sentence in zip(embeddings, faq_pairs)
]


with open('faq_data.json', 'w') as json_file:
    json.dump(sentences_list, json_file, indent=2)

print("JSON file created: faq_data.json")

Processing batches: 0it [00:00, ?it/s]
Processing batch:  25%|██▌       | 5/20 [00:42<03:11, 12.79s/it]
Processing batches: 0it [01:04, ?it/s]                          


RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for text-embedding-3-small in organization org-6TzDDqahD4fc3ws2CSi353gj on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing.', 'type': 'requests', 'param': None, 'code': 'rate_limit_exceeded'}}

In [ ]:
import openai
from openai import OpenAI
from pymilvus import Collection, connections, utility, DataType
import pickle

# Connect to Milvus server
connections.connect(host='localhost', port='19530')

message_history = [{'role': 'user', 'content': '오늘 저녁에 여의도 가려는데 맛집 추천좀 해줄래?'},
                   {'role': 'assistant', 'content': '저는 스마트 스토어 FAQ를 위한 챗봇입니다. 스마트 스토어에 대한 질문을 부탁드립니다.'}]

try:
    while True:
        user_quest = input("\n무엇을 도와드릴까요? (그만하시려면 '끝'을 입력해주세요):\n")

        # Exit loop if user types 'exit'
        if user_quest == '끝':
            break
            
        # Initialize Milvus collection
        faq_collection = Collection(name="faq_data", schema={
            "embeddings": DataType.FLOAT_VECTOR,
            "text": DataType.STRING
        })
        
        # Function to embed text
        def embed_text(text, model="text-embedding-3-small"):
            response = client.embeddings.create(
                input=text,
                model=model
            )
            return response.data[0].embedding
        
        # Function to generate answer
        def generate_answer(messages):
            global message_history
            temp_message_history = message_history.copy()
            temp_message_history.extend(message)
            
            response = client.chat.completion.create(
                model="gpt-3.5-turbo",
                messages=temp_message_history,
                max_tokens=1024,
                n=1,
                stop=None,
                temperature=0.2,
            )
        
            return response.choices[0].message.content
        
        # Function to handle user's question
        def handle_question(user_quest):
            global message_history
            # Embed user's question
            user_question_embedding = embed_text(user_quest)
        
            # Search for relevant FAQ data
            relevant_faq = faq_collection.search(user_question_embedding, anns_field="embeddings", param={}, limit=3, output_fields=["text"])
            relevant_texts = [faq_collection.get_data_by_ids([idx])["text"][0] for idx in relevant_faq]

            # Create initial messages
            messages = []
            messages.extend([{"role": "system", "content": text} for text in relevant_texts])
            messages.extend([{"role": "user", "content": user_quest}])
           
            
            # Generate answer using OpenAI's LLM
            answer = generate_answer(messages)
        
            # Add the question and the generated answer to the message_history
            message_history.extend([{"role": "user", "content": user_quest}])
            message_history.extend([{"role": "assistant", "content": answer}])
        
            return answer
        
        answer = handle_question(user_quest)
        print(answer)

except MilvusException as e:
    # Handle Milvus exceptions
    print(e)

finally:
    # Disconnect from Milvus server
    connections.disconnect(alias="localhost")

In [32]:
import pandas as pd

In [37]:
org = pd.read_csv("embeddings.csv")

In [55]:
org.head(20)

,id,question,answer,question_vector,answer_vector
0,1,[가입절차] 스마트스토어센터 회원가입은 어떻게 하나요? (ID만들기),네이버 커머스 ID 하나로 스마트스토어센터와 같은 네이버의 다양한 커머스 서비스를 ...,"[0.3411521911621094, -0.09419679641723633, 0.0...","[-0.5797623991966248, 0.13396722078323364, -0...."
1,1,[가입절차] 스마트스토어센터 회원가입은 어떻게 하나요? (ID만들기),이메일주소를 입력하시고 인증하신 후 개인정보 수집동의를 체크 하여 [가입] 버튼을 ...,"[0.3411521911621094, -0.09419679641723633, 0.0...","[-0.5505958199501038, -0.12493794411420822, 0...."
2,2,[가입서류] 스마트스토어 판매자 유형별 필요한 서류가 어떻게 되나요?,스마트스토어 판매 회원 분류는 아래와 같으며 반드시 모든 서류를 제출해 주셔야 가입...,"[0.21860308945178986, 0.05236361175775528, 0.1...","[0.07538531720638275, -0.4215330481529236, 0.4..."
3,2,[가입서류] 스마트스토어 판매자 유형별 필요한 서류가 어떻게 되나요?,"부대표자 인감증명서 사본 1부 (3개월 이내 발급분, 생년월일+성별 구분 제외 마스...","[0.21860308945178986, 0.05236361175775528, 0.1...","[-0.4807881712913513, -0.6167274713516235, 0.4..."
4,2,[가입서류] 스마트스토어 판매자 유형별 필요한 서류가 어떻게 되나요?,본 생략 가능· 공동대표의 경우 대표 대표자가 예금주명에 있는 경우 통장 사본 생략...,"[0.21860308945178986, 0.05236361175775528, 0.1...","[-0.17175771296024323, -0.203436940908432, 0.3..."
5,2,[가입서류] 스마트스토어 판매자 유형별 필요한 서류가 어떻게 되나요?,다운로드 받기》스마트스토어 통신판매업 신고번호 입력 기준1. 직전년도 구매확정 수 ...,"[0.21860308945178986, 0.05236361175775528, 0.1...","[0.07784105837345123, 0.06688706576824188, 0.0..."
6,2,[가입서류] 스마트스토어 판매자 유형별 필요한 서류가 어떻게 되나요?,발송)매월 2일마다 안내누적 판매금액 집계 기준(판매자등급 기준과 동일)구매확정 ...,"[0.21860308945178986, 0.05236361175775528, 0.1...","[-0.524160623550415, -0.16115599870681763, 0.5..."
7,2,[가입서류] 스마트스토어 판매자 유형별 필요한 서류가 어떻게 되나요?,"등록증명으로 대체가 가능합니다. 다만, 최근 1년 이내 발급한 서류로 제출 바랍니다...","[0.21860308945178986, 0.05236361175775528, 0.1...","[-0.18004116415977478, -0.7078405022621155, 0...."
8,2,[가입서류] 스마트스토어 판매자 유형별 필요한 서류가 어떻게 되나요?,또는 공증번역기관으로부터 공증을 받은 영문번역본으로 제출해 주셔야 합니다.· 공동대...,"[0.21860308945178986, 0.05236361175775528, 0.1...","[-0.37557199597358704, -0.23363181948661804, 0..."
9,3,[심사결과] 가입 신청을 했습니다. 심사 결과는 어떻게 확인하나요?,"스마트스토어센터에 로그인하시면 ""팝업""또는 [판매자 정보 > 심사내역 조회] 메뉴에...","[-0.3004268407821655, -0.5700594186782837, 0.8...","[0.032250624150037766, 0.1879940927028656, 0.6..."


In [58]:
final = pd.read_csv("modified_combined_embeddings_vectors.csv")

In [59]:
final.head()

,embeddings,vectors
0,[가입절차] 스마트스토어센터 회원가입은 어떻게 하나요? (ID만들기) 네이버 커머스...,"[0.3411521911621094, -0.09419679641723633, 0.0..."
1,[가입서류] 스마트스토어 판매자 유형별 필요한 서류가 어떻게 되나요? 스마트스토어 ...,"[0.21860308945178986, 0.05236361175775528, 0.1..."
2,[심사결과] 가입 신청을 했습니다. 심사 결과는 어떻게 확인하나요? 스마트스토어센터...,"[-0.3004268407821655, -0.5700594186782837, 0.8..."
3,[가입절차][쇼핑윈도/패션타운] 네이버 쇼핑윈도 노출 절차는 어떻게 되나요? 네이버...,"[-0.24794837832450867, -0.22289049625396729, 0..."
4,[가입서류] 국내 사업자로 가입하려면 어떤 서류가 필요한가요? 국내 간이/개인(일반...,"[-0.20949622988700867, -0.612036406993866, 0.4..."


In [63]:
max_length = 0
for i in final['embeddings']:
    if len(i) >= max_length:
        max_length = len(i)
print(max_length)

8974


In [64]:
max_length = 0
for i in final['vectors']:
    if len(i) >= max_length:
        max_length = len(i)
print(max_length)

306705
